In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install sentencepiece

In [ ]:
import random
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, accuracy_score

import tensorflow as tf
import transformers
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, TFAutoModel
from transformers import AdamW

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

import warnings

warnings.filterwarnings("ignore")
os.environ["WANDB_API_KEY"] = "0"

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU

print('Number of replicas:', strategy.num_replicas_in_sync)    

In [ ]:
BATCH_SIZE= 16 * strategy.num_replicas_in_sync

In [ ]:
train = pd.read_csv("/kaggle/input/contradictory-my-dear-watson/train.csv")
test = pd.read_csv("/kaggle/input/contradictory-my-dear-watson/test.csv")

# Lets do a basic EDA

In [ ]:
train['label'].value_counts()
train.shape
train.head()

In [ ]:
print(train['premise'].str.len().describe())
print(train['hypothesis'].str.len().describe())

Look at Test Data

In [ ]:
print(test['premise'].str.len().describe())
print(test['hypothesis'].str.len().describe())

Lets check for Any wrong labels

In [ ]:
wrong_labels = train.groupby(['premise', 'hypothesis']).nunique().sort_values(by='label', ascending=False)
wrong_labels

In [ ]:
train_labels, train_frequencies = np.unique(train.language.values, return_counts = True)
test_labels, test_frequencies = np.unique(test.language.values, return_counts = True)

plt.figure(figsize = (10,10))
plt.pie(train_frequencies,labels = train_labels, autopct = '%1.2f%%')
plt.show()

In [ ]:
plt.figure(figsize = (10,10))
plt.pie(test_frequencies,labels = test_labels, autopct = '%1.2f%%')
plt.show()

There are only 15 languages handled by by this model

In [ ]:
language_list = ['English', 'French', 'Spanish', 'German', 'Greek', 'Bulgarian', 'Russian', 'Turkish', 'Arabic' ,'Vietnamese', 'Thai', 'Chinese', 'Hindi', 'Swahili', 'Urdu']

In [ ]:
for langs in train.language.unique():
    if langs not in language_list:
        print(langs)

In [ ]:
for langs in test.language.unique():
    if langs not in language_list:
        print(langs)

In [ ]:
train_premise_texts, val_premise_texts, train_hypothesis_texts, val_hypothesis_texts, train_labels, val_labels = train_test_split(train['premise'].tolist(), train['hypothesis'].tolist(), train['label'].tolist())

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('joeddav/xlm-roberta-large-xnli')

In [ ]:
train_encodings = tokenizer(train_premise_texts, train_hypothesis_texts,  padding="max_length", truncation=True, return_tensors='tf')
val_encodings = tokenizer(val_premise_texts, val_hypothesis_texts,  padding="max_length", truncation=True, return_tensors='tf')

test_encodings = tokenizer(test['premise'].tolist(), test['hypothesis'].tolist(),  padding="max_length", truncation=True, return_tensors='tf')

In [ ]:
train_tf_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels))
train_tf_dataset = train_tf_dataset.shuffle(len(train_encodings)).batch(BATCH_SIZE)

eval_tf_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), val_labels))
eval_tf_dataset = eval_tf_dataset.batch(BATCH_SIZE)

In [ ]:
try_encodings = tokenizer(train['premise'].tolist()[:3], train['hypothesis'].tolist()[:3],  padding="max_length", truncation=True, return_tensors='tf')

In [ ]:
MAX_LEN = train_encodings['input_ids'].shape[1]

with strategy.scope():
    input_ids = tf.keras.Input(shape =(512,), dtype=tf.int32, name='input_ids') 
    attention_mask = tf.keras.Input(shape=(512,),dtype=tf.int32, name='attention_mask')  
    
    roberta = TFAutoModel.from_pretrained('joeddav/xlm-roberta-large-xnli')
    
    for layers in roberta.layers:
        layers.trainable = False
        
    tf_model_output = roberta([input_ids, attention_mask])[1]
    
    tf_model_output = tf.keras.layers.Dense(512, activation = 'relu')(tf_model_output)
    tf_model_output = tf.keras.layers.Dropout(0.2)(tf_model_output)
    tf_model_output = tf.keras.layers.Dense(256, activation = 'relu')(tf_model_output)
    tf_model_output = tf.keras.layers.Dropout(0.1)(tf_model_output)
    tf_model_output = tf.keras.layers.Dense(64, activation = 'relu')(tf_model_output)
    output = tf.keras.layers.Dense(3, activation = 'softmax')(tf_model_output)
        
    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)
    
    model.compile(optimizer = tf.keras.optimizers.Adam(lr=1e-3), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy']) 
    
    model.summary()

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

model.fit(train_tf_dataset, 
        validation_data=eval_tf_dataset,
        callbacks = [early_stop],
        epochs=20,
        verbose=1)


# Lets fine tune the whole model now

In [ ]:
for layers in roberta.layers:
    layers.trainable = True
model.compile(optimizer = tf.keras.optimizers.Adam(lr=3e-5), 
                  loss='sparse_categorical_crossentropy',  
                  metrics=['accuracy'])

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

model.fit(train_tf_dataset, 
        validation_data=eval_tf_dataset,
        callbacks = [early_stop],
        epochs=8,
        verbose=1)

In [ ]:
test = pd.read_csv('../input/contradictory-my-dear-watson/test.csv')

In [ ]:
test_encoded = tokenizer(test['premise'].tolist(), test['hypothesis'].tolist(), padding='max_length', return_tensors='tf')
predictions = [np.argmax(i) for i in model.predict(test_encoded.data)]

In [ ]:
submission = test.id.copy().to_frame()
submission['prediction'] = predictions
print(submission['prediction'].value_counts())
print(submission.head())

In [ ]:
submission.to_csv("submission.csv", index=False)